<a href="https://colab.research.google.com/github/qizhangumich/Amazon-Recommendation-System-Capstone/blob/master/JZ_Youtube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("Notebook started")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 设置文件路径
CLIENT_SECRETS_FILE = '/content/drive/My Drive/my project/youtube/client_secret.json'
TOKEN_FILE = '/content/drive/My Drive/my project/youtube/token.pickle'

In [ ]:
import os
import pickle
import google.auth.transport.requests
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors
import smtplib
from email.mime.text import MIMEText
from datetime import datetime, timedelta
import pytz  # 处理时区
from google.colab import drive  # 适用于 Google Colab

# 挂载 Google Drive
#drive.mount('/content/drive')

# 配置文件路径
#CLIENT_SECRETS_FILE = '/content/drive/My Drive/OAuth/client_secret.json'  # 在 Google Drive 中的路径
#TOKEN_FILE = '/content/drive/My Drive/OAuth/token.pickle'  # 在 Google Drive 中存储 OAuth 凭证
SCOPES = ["https://www.googleapis.com/auth/youtube.readonly"]

# 你需要检查的频道 ID 列表
channel_ids = [
    'UCESh5daDcPqvG0QhbSdzIzw',  # Millionaires Investment Secrets
    'UCbmNph6atAoGfqLoCL_duAg',  # Talks at Google
    'UCp5hG8rt1z2MJ9aNVxY2Xdg',  # New Economic Thinking
    'UCYO_jab_esuFRV4b17AJtAw',  # 3Blue1Brown
    'UCQ58rNgEE1Vt3G40E_NWAsA',  # 每日一书
    'UC2L7vR43LKuBXXV2AentEMw',  # Luke's English Podcast
    'UC6LU7FUBvbFCh_cQasrHZ_Q',  # MacroMicro財經M平方
    'UCZgt6AzoyjslHTC9dz0UoTw',  # ByteByteGo
    'UCy32uaK6T0itKPG5mtH9sow',  # Math Window
    'UCO17pDLFQBKGitI2F7_c_JQ',  # Alanis Business Academy
    'UCpCSAcbqs-sjEVfk_hMfY9w',  # Zach Star
    'UCd6Za0CXVldhY8fK8eYoIuw',  # Freedom in Thought
    'UC2m77KGMIKZCD0Kw35-4shQ',  # Yuval Noah Harari
    'UC-98P1HknuXGxDYNwlxsKwQ'   # The Great Courses
]

# 获取已认证的 YouTube 服务
def get_authenticated_service():
    credentials = None

    # 尝试从 Google Drive 加载保存的凭据
    if os.path.exists(TOKEN_FILE):
        with open(TOKEN_FILE, 'rb') as token_file:
            credentials = pickle.load(token_file)

    # 如果没有凭据或凭据无效，则通过 OAuth 流程授权
    if not credentials or not credentials.valid:
        if credentials and credentials.expired and credentials.refresh_token:
            # 凭据过期但有刷新令牌，则刷新凭据
            credentials.refresh(google.auth.transport.requests.Request())
        else:
            # 第一次授权，启动 OAuth 流程
            flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
            credentials = flow.run_local_server(port=0, open_browser=False)

        # 将新凭据保存到 Google Drive
        with open(TOKEN_FILE, 'wb') as token_file:
            pickle.dump(credentials, token_file)

    return googleapiclient.discovery.build("youtube", "v3", credentials=credentials)

# 获取频道上传视频的播放列表 ID
def get_uploads_playlist_id(youtube, channel_id):
    try:
        request = youtube.channels().list(
            part="contentDetails",
            id=channel_id
        )
        response = request.execute()
        if 'items' in response and len(response['items']) > 0:
            uploads_playlist_id = response['items'][0]['contentDetails']['relatedPlaylists']['uploads']
            return uploads_playlist_id
        else:
            print(f"未找到与频道 ID {channel_id} 相关的内容。可能是无效的频道 ID。")
            return None
    except googleapiclient.errors.HttpError as e:
        print(f"获取频道 {channel_id} 时发生错误: {e}")
        return None

# 获取播放列表中最近 24 小时发布的视频
def get_latest_videos_from_playlist(youtube, playlist_id, max_results=5):
    try:
        request = youtube.playlistItems().list(
            part="snippet",
            playlistId=playlist_id,
            maxResults=max_results
        )
        response = request.execute()
        print("YouTube information collected")
        videos = []
        if 'items' in response:
            for item in response['items']:
                video_title = item['snippet']['title']
                video_id = item['snippet']['resourceId']['videoId']
                published_at = item['snippet']['publishedAt']
                video_link = f"https://www.youtube.com/watch?v={video_id}"

                # 转换发布时间为 datetime 对象
                published_at_dt = datetime.strptime(published_at, "%Y-%m-%dT%H:%M:%SZ")
                published_at_dt = pytz.utc.localize(published_at_dt)  # 将其设置为 UTC 时间

                # 获取当前时间并计算过去 24 小时
                now = datetime.now(pytz.utc)
                past_24_hours = now - timedelta(hours=24)

                # 只保留过去 24 小时发布的视频
                if published_at_dt > past_24_hours:
                    videos.append((video_title, video_link, published_at))

        return videos
    except googleapiclient.errors.HttpError as e:
        print(f"获取播放列表 {playlist_id} 时发生错误: {e}")
        return []

# 发送邮件提醒
def send_email(subject, body, to_email):
    msg = MIMEText(body)
    msg['Subject'] = subject
    msg['From'] = 'zhangqi362@163.com'  # 发件人
    msg['To'] = to_email  # 收件人

    # 使用 163 邮箱的 SMTP 服务器发送邮件
    smtp_server = 'smtp.163.com'
    smtp_port = 25  # 163 邮箱使用端口 25
    sender_email = 'zhangqi362@163.com'
    sender_password = 'QPGGDKBLTZIYWDMZ'  # 163 邮箱的授权码或密码

    try:
        # 使用 SMTP 连接发送邮件
        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.starttls()  # 启用 TLS 加密
            server.login(sender_email, sender_password)  # 登录邮箱
            server.sendmail(sender_email, to_email, msg.as_string())  # 发送邮件
        print(f"邮件已成功发送至 {to_email}")
    except Exception as e:
        print(f"发送邮件时出错: {e}")

# 检查所有频道的视频更新，并汇总后发送一封邮件
def check_all_channels_and_notify(youtube):
    all_videos = []  # 用于存储所有新视频信息

    for channel_id in channel_ids:
        print(f"检查频道: {channel_id}")
        uploads_playlist_id = get_uploads_playlist_id(youtube, channel_id)
        if uploads_playlist_id:
            latest_videos = get_latest_videos_from_playlist(youtube, uploads_playlist_id)
            if latest_videos:
                all_videos.extend(latest_videos)

    # 如果有新视频，汇总并发送邮件
    if all_videos:
        body = "\n\n".join([f"视频标题: {title}\n链接: {link}\n发布时间: {published_at}" for title, link, published_at in all_videos])
        send_email("每日You最新讯息", body, 'zhangqi@cpe-fund.com')
        print("Email sent successfully")
    else:
        print("没有过去 24 小时内的新视频发布。")

# 主程序
def main():
    youtube = get_authenticated_service()
    check_all_channels_and_notify(youtube)

if __name__ == "__main__":
    main()


检查频道: UCESh5daDcPqvG0QhbSdzIzw
检查频道: UCbmNph6atAoGfqLoCL_duAg
检查频道: UCp5hG8rt1z2MJ9aNVxY2Xdg
检查频道: UCYO_jab_esuFRV4b17AJtAw
检查频道: UCQ58rNgEE1Vt3G40E_NWAsA
检查频道: UC2L7vR43LKuBXXV2AentEMw
检查频道: UC6LU7FUBvbFCh_cQasrHZ_Q
检查频道: UCZgt6AzoyjslHTC9dz0UoTw
检查频道: UCy32uaK6T0itKPG5mtH9sow
检查频道: UCO17pDLFQBKGitI2F7_c_JQ
检查频道: UCpCSAcbqs-sjEVfk_hMfY9w
检查频道: UCd6Za0CXVldhY8fK8eYoIuw
检查频道: UC2m77KGMIKZCD0Kw35-4shQ
检查频道: UC-98P1HknuXGxDYNwlxsKwQ
邮件已成功发送至 zhangqi@cpe-fund.com
